# Convert Card Data to i18n Format

This notebook converts the structured card data to a format suitable for i18n in a Nuxt.js project, with English field names as keys.

In [9]:
import json
import re
from pathlib import Path

In [10]:
# Define mapping of Japanese field names to English field names
field_mapping = {
    # Card basic info
    'カードタイプ': 'cardType',
    'タグ': 'tags',
    'レアリティ': 'rarity',
    '収録商品': 'set',
    'カードナンバー': 'cardNumber',
    'イラストレーター': 'illustrator',
    
    # Card attributes
    '色': 'color',
    'HP': 'hp',
    'LIFE': 'life',
    'Bloomレベル': 'bloomLevel',
    'バトンタッチ': 'batonTouch',
    
    # Card abilities
    'アーツ': 'arts',
    'エクストラ': 'extra',
    'キーワード': 'keyword',
    '推しスキル': 'oshiSkill',
    'SP推しスキル': 'spOshiSkill',
    '能力テキスト': 'abilityText',
    
    # Arts and skill components
    'name': 'name',
    'effect': 'effect',
    'full_text': 'fullText',
    'damage': 'damage',
    'cost': 'cost',
    'timing': 'timing',
    'count': 'count',
    'images': 'images',
    'cost_icons': 'costIcons',
    'tokkou': 'special',
    'icon': 'icon',
    'value': 'value'
}

# Define bloom level mapping
bloom_level_mapping = {
    'Debut': 'debut',
    '1st': 'first',
    '2nd': 'second'
}

# Define color mapping
color_mapping = {
    '白': 'white',
    '赤': 'red',
    '青': 'blue',
    '緑': 'green',
    '黄': 'yellow',
    '紫': 'purple'
}

# Card type mapping
card_type_mapping = {
    'ホロメン': 'character',
    'Buzzホロメン': 'buzzCharacter',
    '推しホロメン': 'oshiCharacter',
    'サポート・イベント': 'supportEvent',
    'サポート・イベント・LIMITED': 'supportEventLimited',
    'サポート・アイテム': 'supportItem',
    'サポート・ファン': 'supportFan',
    'サポート・ツール': 'supportTool',
    'サポート・ロケーション': 'supportLocation',
    'サポート・エール': 'supportCheer'
}

In [11]:
def convert_card_to_i18n(card, default_language='ja'):
    """Convert a card's structured data to i18n format with English keys."""
    i18n_card = {
        'id': card.get('id', ''),
        'translations': {}
    }
    
    # Initialize translations for the default language
    translations = {}
    
    # Process card number and image paths if available
    if 'カードナンバー' in card:
        i18n_card['cardNumber'] = card['カードナンバー']
        
    if 'image_path' in card:
        i18n_card['imagePath'] = card['image_path']
    
    if 'image_url' in card:
        i18n_card['imageUrl'] = card['image_url']
        
    # Process name separately as it's often at the top level
    if 'name' in card:
        translations['name'] = card['name']
    
    # Process card type
    if 'カードタイプ' in card:
        card_type_ja = card['カードタイプ']
        translations['cardType'] = card_type_ja
        i18n_card['cardTypeCode'] = card_type_mapping.get(card_type_ja, 'unknown')
    
    # Process card color
    if '色' in card:
        if isinstance(card['色'], dict) and 'value' in card['色']:
            color_ja = card['色']['value']
            translations['color'] = color_ja
            i18n_card['colorCode'] = color_mapping.get(color_ja, 'unknown')
        elif isinstance(card['色'], str):
            color_ja = card['色']
            translations['color'] = color_ja
            i18n_card['colorCode'] = color_mapping.get(color_ja, 'unknown')
    
    # Process bloom level
    if 'Bloomレベル' in card:
        bloom_level_ja = card['Bloomレベル']
        translations['bloomLevel'] = bloom_level_ja
        i18n_card['bloomLevelCode'] = bloom_level_mapping.get(bloom_level_ja, 'unknown')
    
    # Process numeric values directly
    for jp_field, en_field in [
        ('HP', 'hp'), 
        ('LIFE', 'life')
    ]:
        if jp_field in card:
            try:
                i18n_card[en_field] = int(card[jp_field])
            except (ValueError, TypeError):
                i18n_card[en_field] = card[jp_field]
    
    # Process tags
    if 'タグ' in card and isinstance(card['タグ'], list):
        tags = []
        tag_translations = []
        
        for tag in card['タグ']:
            if isinstance(tag, dict) and 'name' in tag:
                tag_name = tag['name']
                tag_code = tag_name.replace('#', '')
                tags.append(tag_code)
                tag_translations.append(tag_name)
            elif isinstance(tag, str):
                tag_code = tag.replace('#', '')
                tags.append(tag_code)
                tag_translations.append(tag)
        
        i18n_card['tags'] = tags
        translations['tags'] = tag_translations
    
    # Process rarity
    if 'レアリティ' in card:
        translations['rarity'] = card['レアリティ']
        i18n_card['rarityCode'] = card['レアリティ']
    
    # Process set
    if '収録商品' in card:
        translations['set'] = card['収録商品']
    
    # Process illustrator
    if 'イラストレーター' in card:
        translations['illustrator'] = card['イラストレーター']
    
    # Process ability text for support cards
    if '能力テキスト' in card:
        translations['abilityText'] = card['能力テキスト']
    
    # Process extra text
    if 'エクストラ' in card:
        translations['extra'] = card['エクストラ']
    
    # Process baton touch
    if 'バトンタッチ' in card:
        if isinstance(card['バトンタッチ'], dict) and 'count' in card['バトンタッチ']:
            i18n_card['batonTouchCount'] = card['バトンタッチ']['count']
    
    # Process arts
    if 'アーツ' in card and isinstance(card['アーツ'], list):
        arts_list = []
        arts_translations = []
        
        for i, arts in enumerate(card['アーツ']):
            arts_data = {}
            arts_translation = {}
            
            # Handle cost icons
            if 'cost_icons' in arts and isinstance(arts['cost_icons'], list):
                cost_count = len(arts['cost_icons'])
                arts_data['costCount'] = cost_count
                
                # Extract cost types based on alt text
                cost_types = []
                for icon in arts['cost_icons']:
                    if isinstance(icon, dict) and 'alt' in icon:
                        alt = icon['alt']
                        if alt in color_mapping:
                            cost_types.append(color_mapping[alt])
                        elif alt == '◇':
                            cost_types.append('neutral')
                arts_data['costTypes'] = cost_types
            
            # Handle arts name
            if 'name' in arts:
                arts_translation['name'] = arts['name']
            
            # Handle damage value
            if 'damage' in arts:
                try:
                    # Extract base damage without '+' if present
                    damage_str = arts['damage']
                    base_damage = re.match(r'(\d+)', damage_str)
                    if base_damage:
                        arts_data['damage'] = int(base_damage.group(1))
                    
                    # Check if it's a plus damage
                    if '+' in damage_str:
                        arts_data['isPlus'] = True
                except (ValueError, TypeError):
                    arts_data['damage'] = arts['damage']
            
            # Handle special damage
            if 'tokkou' in arts and isinstance(arts['tokkou'], list):
                special_targets = []
                special_values = []
                
                for special in arts['tokkou']:
                    if isinstance(special, dict) and 'alt' in special:
                        alt_text = special['alt']
                        # Parse special damage like "赤+50"
                        match = re.match(r'([^+]+)\+(\d+)', alt_text)
                        if match:
                            color_name = match.group(1)
                            damage_value = int(match.group(2))
                            if color_name in color_mapping:
                                target = color_mapping[color_name]
                                special_targets.append(target)
                                special_values.append(damage_value)
                
                if special_targets:
                    arts_data['specialTargets'] = special_targets
                    arts_data['specialValues'] = special_values
            
            # Handle effect text
            if 'effect' in arts:
                arts_translation['effect'] = arts['effect']
                
            # Add to lists
            arts_list.append(arts_data)
            arts_translations.append(arts_translation)
        
        i18n_card['arts'] = arts_list
        translations['arts'] = arts_translations
    
    # Process keyword abilities
    if 'キーワード' in card and isinstance(card['キーワード'], dict):
        keyword = card['キーワード']
        keyword_data = {}
        keyword_translation = {}
        
        # Extract keyword type from icon
        if 'icon' in keyword and isinstance(keyword['icon'], dict) and 'alt' in keyword['icon']:
            keyword_type = keyword['icon']['alt']
            keyword_data['type'] = keyword_type
            
            # Map keyword types to codes
            keyword_type_code = keyword_type.lower()
            if 'コラボエフェクト' in keyword_type:
                keyword_type_code = 'collab_effect'
            elif 'ブルームエフェクト' in keyword_type:
                keyword_type_code = 'bloom_effect'
            elif 'ギフト' in keyword_type:
                keyword_type_code = 'gift'
            
            keyword_data['typeCode'] = keyword_type_code
        
        # Extract name and effect
        if 'name' in keyword:
            keyword_translation['name'] = keyword['name']
        
        if 'effect' in keyword:
            keyword_translation['effect'] = keyword['effect']
        
        i18n_card['keyword'] = keyword_data
        translations['keyword'] = keyword_translation
    
    # Process oshi skill
    for skill_type, en_field in [('推しスキル', 'oshiSkill'), ('SP推しスキル', 'spOshiSkill')]:
        if skill_type in card and isinstance(card[skill_type], dict):
            skill = card[skill_type]
            skill_data = {}
            skill_translation = {}
            
            # Extract cost
            if 'cost' in skill:
                try:
                    # Cost is usually a negative number like "-2"
                    skill_data['cost'] = int(skill['cost'])
                except (ValueError, TypeError):
                    skill_data['cost'] = skill['cost']
            
            # Extract timing
            if 'timing' in skill:
                timing_text = skill['timing']
                skill_translation['timing'] = timing_text
                
                # Map timing to code
                timing_code = 'unknown'
                if 'ターンに1回' in timing_text:
                    timing_code = 'once_per_turn'
                elif 'ゲームに1回' in timing_text:
                    timing_code = 'once_per_game'
                
                skill_data['timingCode'] = timing_code
            
            # Extract name and effect
            if 'name' in skill:
                skill_translation['name'] = skill['name']
            
            if 'effect' in skill:
                skill_translation['effect'] = skill['effect']
            
            i18n_card[en_field] = skill_data
            translations[en_field] = skill_translation
    
    # Add translations to the card
    i18n_card['translations'][default_language] = translations
    
    return i18n_card

In [12]:
def generate_card_i18n_data(input_file='./cards_structured.json', output_file='./cards_i18n.json'):
    """Process all structured card data and convert to i18n format."""
    # Load structured data
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            cards_data = json.load(f)
    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
        return
    except json.JSONDecodeError:
        print(f"Error: Input file '{input_file}' contains invalid JSON.")
        return
    
    # Convert each card to i18n format
    i18n_cards = []
    for i, card in enumerate(cards_data):
        print(f"Processing card {i+1}/{len(cards_data)}")
        i18n_card = convert_card_to_i18n(card)
        i18n_cards.append(i18n_card)
    
    # Save the result
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(i18n_cards, f, ensure_ascii=False, indent=2)
        print(f"Success: Converted {len(i18n_cards)} cards to i18n format. Saved to '{output_file}'.")
    except Exception as e:
        print(f"Error saving output file: {str(e)}")
    
    return i18n_cards

In [13]:
# Generate card i18n data
i18n_cards = generate_card_i18n_data()

Processing card 1/845
Processing card 2/845
Processing card 3/845
Processing card 4/845
Processing card 5/845
Processing card 6/845
Processing card 7/845
Processing card 8/845
Processing card 9/845
Processing card 10/845
Processing card 11/845
Processing card 12/845
Processing card 13/845
Processing card 14/845
Processing card 15/845
Processing card 16/845
Processing card 17/845
Processing card 18/845
Processing card 19/845
Processing card 20/845
Processing card 21/845
Processing card 22/845
Processing card 23/845
Processing card 24/845
Processing card 25/845
Processing card 26/845
Processing card 27/845
Processing card 28/845
Processing card 29/845
Processing card 30/845
Processing card 31/845
Processing card 32/845
Processing card 33/845
Processing card 34/845
Processing card 35/845
Processing card 36/845
Processing card 37/845
Processing card 38/845
Processing card 39/845
Processing card 40/845
Processing card 41/845
Processing card 42/845
Processing card 43/845
Processing card 44/8

In [14]:
# Generate separate language files for Nuxt.js i18n setup
def generate_language_files(i18n_cards, output_dir='../locales'):
    """Generate separate language files for Nuxt.js i18n setup."""
    # Create output directory if it doesn't exist
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Collect all translations by language
    translations_by_lang = {}
    
    for card in i18n_cards:
        for lang, translations in card.get('translations', {}).items():
            if lang not in translations_by_lang:
                translations_by_lang[lang] = {'cards': {}}
            
            card_id = card.get('id', '') or card.get('cardNumber', '')
            if card_id:
                translations_by_lang[lang]['cards'][card_id] = translations
    
    # Save each language file
    for lang, content in translations_by_lang.items():
        lang_file = output_path / f"{lang}.json"
        try:
            with open(lang_file, 'w', encoding='utf-8') as f:
                json.dump(content, f, ensure_ascii=False, indent=2)
            print(f"Language file saved: {lang_file}")
        except Exception as e:
            print(f"Error saving language file {lang}: {str(e)}")
    
    # Generate an English translation template based on Japanese
    if 'ja' in translations_by_lang:
        english_translations = {'cards': {}}
        
        for card_id, ja_translations in translations_by_lang['ja']['cards'].items():
            card_en = {}
            
            # Generate placeholder English translations
            for key, value in ja_translations.items():
                if isinstance(value, list):
                    # Handle nested lists like tags and arts
                    if key == 'tags':
                        card_en[key] = [f"[EN] {tag}" for tag in value]
                    elif key == 'arts':
                        arts_en = []
                        for art in value:
                            art_en = {}
                            for art_key, art_value in art.items():
                                art_en[art_key] = f"[EN] {art_value}"
                            arts_en.append(art_en)
                        card_en[key] = arts_en
                elif isinstance(value, dict):
                    # Handle nested dictionaries like keyword and skills
                    nested_en = {}
                    for nested_key, nested_value in value.items():
                        nested_en[nested_key] = f"[EN] {nested_value}"
                    card_en[key] = nested_en
                else:
                    # Regular string values
                    card_en[key] = f"[EN] {value}"
            
            english_translations['cards'][card_id] = card_en
        
        # Save English template
        en_file = output_path / "en.json"
        try:
            with open(en_file, 'w', encoding='utf-8') as f:
                json.dump(english_translations, f, ensure_ascii=False, indent=2)
            print(f"English template file saved: {en_file}")
        except Exception as e:
            print(f"Error saving English template: {str(e)}")

In [15]:
# Generate language files
if i18n_cards:
    generate_language_files(i18n_cards)

Language file saved: ../locales/ja.json
English template file saved: ../locales/en.json


## How to Use in Nuxt.js

1. Install the i18n module:
```bash
npm install @nuxtjs/i18n
```

2. Add the i18n module to your `nuxt.config.ts`:
```typescript
export default defineNuxtConfig({
  modules: [
    '@nuxtjs/i18n',
  ],
  i18n: {
    locales: [
      { code: 'en', iso: 'en-US', file: 'en.json', name: 'English' },
      { code: 'ja', iso: 'ja-JP', file: 'ja.json', name: '日本語' },
    ],
    defaultLocale: 'ja',
    lazy: true,
    langDir: 'locales/',
    strategy: 'prefix_except_default',
  },
})
```

3. Access card translations in your Nuxt components:
```vue
<template>
  <div>
    <h1>{{ $t(`cards.${cardId}.name`) }}</h1>
    <p>{{ $t(`cards.${cardId}.cardType`) }}</p>
    <!-- For nested structures like arts -->
    <div v-for="(art, index) in $t(`cards.${cardId}.arts`)" :key="index">
      <h3>{{ art.name }}</h3>
      <p>{{ art.effect }}</p>
    </div>
  </div>
</template>

<script setup>
const { cardId } = defineProps(['cardId'])
</script>
```